# Getting going with MagIC and rockmagpy

## Background on MagIC

The Magnetics Information Consortium (MagIC), hosted at http://earthref.org/MagIC is a database that serves as a Findable, Accessible, Interoperable, Reusable (FAIR) archive for paleomagnetic and rock magnetic data. Its data model is fully described here: [https://www2.earthref.org/MagIC/data-models/3.0](https://www2.earthref.org/MagIC/data-models/3.0). Each contribution is associated with a publication via the DOI.  

There are nine data tables. The fields associated with these tables are detailed here: https://www2.earthref.org/MagIC/data-models/3.0

- contribution: metadata of the associated publication.
- locations: metadata for locations, which are groups of sites (e.g., stratigraphic section, region, etc.)
- sites: metadata and derived data at the site level (units with a common expectation)
- samples: metadata and derived data at the sample level.
- specimens: metadata and derived data at the specimen level.
- measurements: metadata and measurements at the specimen level.
- criteria: criteria by which data are deemed acceptable
- ages: ages and metadata for sites/samples/specimens
- images: associated images and plots.

## Importing data from MagIC

This notebook illustrates the two approaches that can be taken to import data from MagIC into a Jupyter notebook:
- **From a locally hosted (i.e. downloaded) MagIC contribution**
- **Directly from the MagIC database (using the API)**
  - either from a public contribution or
  - from a private contribution

The MagIC file is then unpacked into its constituent files and loaded as a contribution object. The notebook then summarizes the specimens and the experiments that have been run on them within the data. Finally, it is illustrated how data from a single specimen and single experiment on that specimen can be isolated.

In the other rockmagpy notebooks, these import methods are used to bring in contribution data. The filtering methods to access data from specific specimens and experiments are used either in the notebooks or within the `rockmag.py` functions.

## Import scientific python libraries

Run the cell below to import the functions needed for the notebook including `rockmagpy` which is imported as `import pmagpy.rockmag as rmag`.

In [1]:
import pmagpy.ipmag as ipmag
import pmagpy.rockmag as rmag
import pmagpy.contribution_builder as cb
import pandas as pd
pd.set_option('display.max_columns', 100)
from bokeh.io import output_notebook
output_notebook(hide_banner=True)

## Unpack data from a local MagIC file

Within the folder `./example_data/ECMB`, there is a MagIC contribution called `'magic_contribution_20213.txt'` that has been downloaded already from the MagIC database and is associated with this publication:

> Swanson-Hysell, N.L., Avery, M.S., Zhang, Y., Hodgin, E.B., Sherwood, R.J., Apen, F.E., Boerboom, T.J., Keller, C.B., and Cottle, J.M. (2021), The paleogeography of Laurentia in its early years: new constraints from the Paleoproterozoic East Central Minnesota batholith, Tectonics, doi:10.1130/G47873.1.

 Running the code cell below unpacks that MagIC contribution so that we can visualize and analyze the measurement data. We can then create a contribution object that has all of the MagIC tables. This approach can be applied to MagIC contributions that are downloaded from the database.

In [3]:
# set the directory path (dir_path) to the data
dir_path = './example_data/ECMB'

# set the name of the MagIC file
ipmag.unpack_magic('magic_contribution_20213.txt', 
                     dir_path = dir_path,
                     input_dir_path = dir_path,
                     print_progress=False)

ECMB_contribution = cb.Contribution(dir_path)

1  records written to file  /Users/penokean/0000_GitHub/2025_rockmagpy_workshop/2_MagIC_hysteresis/example_data/ECMB/contribution.txt
1  records written to file  /Users/penokean/0000_GitHub/2025_rockmagpy_workshop/2_MagIC_hysteresis/example_data/ECMB/locations.txt
90  records written to file  /Users/penokean/0000_GitHub/2025_rockmagpy_workshop/2_MagIC_hysteresis/example_data/ECMB/sites.txt
312  records written to file  /Users/penokean/0000_GitHub/2025_rockmagpy_workshop/2_MagIC_hysteresis/example_data/ECMB/samples.txt
1574  records written to file  /Users/penokean/0000_GitHub/2025_rockmagpy_workshop/2_MagIC_hysteresis/example_data/ECMB/specimens.txt
17428  records written to file  /Users/penokean/0000_GitHub/2025_rockmagpy_workshop/2_MagIC_hysteresis/example_data/ECMB/measurements.txt
-I- Using online data model
-I- Getting method codes from earthref.org
-I- Importing controlled vocabularies from https://earthref.org


### Inspect the measurements table

Within the MagIC contribution object, there is a measurements table that we can inspect.

In [4]:
ECMB_measurements = ECMB_contribution.tables['measurements'].df
ECMB_measurements.dropna(axis=1, how='all').tail(2) # see the last 2 measurements without empty columns

,citations,dir_csd,dir_dec,dir_inc,experiment,instrument_codes,magn_mass,magn_moment,magn_r2_det,magn_x_sigma,magn_y_sigma,magn_z_sigma,meas_field_dc,meas_n_orient,meas_temp,measurement,method_codes,quality,sequence,software_packages,specimen,standard,timestamp,treat_ac_field,treat_dc_field,treat_dc_field_phi,treat_dc_field_theta,treat_step_num,treat_temp
measurement name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
NED4-1c-LP-CW-SIRM:LP-MW-DC-13024-3-57,None,NaN,NaN,NaN,NED4-1c-LP-CW-SIRM:LP-MW-DC-13024,IRM-OldBlue,0.001566,NaN,0.987632,NaN,NaN,NaN,0.0,NaN,295.6190,NED4-1c-LP-CW-SIRM:LP-MW-DC-13024-3-57,LP-CW-SIRM:LP-MW,g,17427,None,NED4-1c,u,2020:01:07:03:00:42.762,NaN,NaN,NaN,NaN,NaN,NaN
NED4-1c-LP-CW-SIRM:LP-MW-DC-13024-3-58,None,NaN,NaN,NaN,NED4-1c-LP-CW-SIRM:LP-MW-DC-13024,IRM-OldBlue,0.001534,NaN,0.987948,NaN,NaN,NaN,0.0,NaN,300.0338,NED4-1c-LP-CW-SIRM:LP-MW-DC-13024-3-58,LP-CW-SIRM:LP-MW,g,17428,None,NED4-1c,u,2020:01:07:03:07:06.72,NaN,NaN,NaN,NaN,NaN,NaN


### List out the experiments

We can use the function `rmag.make_experiment_df()` to list out the experiments that are a part of this MagIC contribution. We will exclude the experiments that are associated with AF and thermal demagnetization to obtain directional magnetization data in order to focus on the hysteresis experiments and low-temperature experiments.

The method codes associated with the experiments are from the controlled vocabularies in the MagIC database https://www2.earthref.org/MagIC/method-codes.

| Method Code    | Definition |
| -------------- | ---------- |
| LP-BCR-BF      | Coercivity of remanence: Back field method | 
| LP-HYS | Hysteresis loops |
| LP-CW-SIRM:LP-MC| Cycling between cooling and warming: Room temperature SIRM; Measured while cooling |
| LP-CW-SIRM:LP-MW| Cycling between cooling and warming: Room temperature SIRM; Measured while warming |
| LP-FC          | Field cooled: Remanent magnetization measured on warming |
| LP-ZFC         | Zero field cooled: Remanent magnetization measured on warming |

In [5]:
ECMB_experiments = rmag.make_experiment_df(ECMB_measurements,
                                           exclude_method_codes=['LP-DIR-AF','LP-DIR-T'])
ECMB_experiments

,specimen,method_codes,experiment
0,NED1-5c,LP-BCR-BF,IRM-VSM3-LP-BCR-BF-218846
1,NED1-5c,LP-HYS,IRM-VSM3-LP-HYS-218845
2,NED18-2c,LP-BCR-BF,IRM-VSM3-LP-BCR-BF-218848
3,NED18-2c,LP-CW-SIRM:LP-MC,NED18-2c-LP-CW-SIRM:LP-MC-DC-13025
4,NED18-2c,LP-CW-SIRM:LP-MW,NED18-2c-LP-CW-SIRM:LP-MW-DC-13025
5,NED18-2c,LP-FC,NED18-2c-LP-FC-DC-13025
6,NED18-2c,LP-HYS,IRM-VSM3-LP-HYS-218847
7,NED18-2c,LP-ZFC,NED18-2c-LP-ZFC-DC-13025
8,NED2-8c,LP-BCR-BF,IRM-VSM3-LP-BCR-BF-218850
9,NED2-8c,LP-CW-SIRM:LP-MC,NED2-8c-LP-CW-SIRM:LP-MC-DC-13038


### Analyze and plot a hysteresis loop

In [19]:
#specify the experiment name and specimen name
experiment_name = 'IRM-VSM3-LP-HYS-218845'
specimen_name = 'NED1-5c'

experiment_hyst = rmag.experiment_selection(ECMB_measurements,
                                            experiment_name)
experiment_results = rmag.process_hyst_loop(experiment_hyst['meas_field_dc'].values, 
                                            experiment_hyst['magn_mass'].values, specimen_name)

NameError: name 'ECMB_measurements' is not defined

### Plot low-temperature remanence experiments

In [9]:
specimen_name = 'NED4-1c'
fc_data, zfc_data, rtsirm_cool_data, rtsirm_warm_data = rmag.extract_mpms_data_dc(ECMB_measurements, specimen_name)
rmag.plot_mpms_dc(fc_data, zfc_data, rtsirm_cool_data, rtsirm_warm_data, 
                  interactive=True, plot_derivative=True)

## Direct download of a published MagIC contribution

We can directly download a public MagIC contribution by providing the MagIC ID to the function `ipmag.download_magic_from_id()`.

Let's do that and pull in Rock Magnetic Bestiary data associated with oxyhydroxides that have been published in the MagIC database here: https://earthref.org/MagIC/20427 with ```'20427'``` being the MagIC ID.

In [11]:
# set the MagIC ID for the data set here
magic_id = '20427'

# set where you want the downloaded data to go
dir_path = 'example_data/RMB_oxyhydroxides'

result, magic_file = ipmag.download_magic_from_id(magic_id, directory=dir_path)
ipmag.unpack_magic(magic_file, dir_path, print_progress=False)
oxy_contribution = cb.Contribution(dir_path)
oxy_measurements = oxy_contribution.tables['measurements'].df

Download successful. File saved to: example_data/RMB_oxyhydroxides/magic_contribution_20427.txt
1  records written to file  /Users/penokean/0000_GitHub/2025_rockmagpy_workshop/2_MagIC_hysteresis/example_data/RMB_oxyhydroxides/contribution.txt
6  records written to file  /Users/penokean/0000_GitHub/2025_rockmagpy_workshop/2_MagIC_hysteresis/example_data/RMB_oxyhydroxides/locations.txt
6  records written to file  /Users/penokean/0000_GitHub/2025_rockmagpy_workshop/2_MagIC_hysteresis/example_data/RMB_oxyhydroxides/sites.txt
84  records written to file  /Users/penokean/0000_GitHub/2025_rockmagpy_workshop/2_MagIC_hysteresis/example_data/RMB_oxyhydroxides/samples.txt
85  records written to file  /Users/penokean/0000_GitHub/2025_rockmagpy_workshop/2_MagIC_hysteresis/example_data/RMB_oxyhydroxides/specimens.txt
3800  records written to file  /Users/penokean/0000_GitHub/2025_rockmagpy_workshop/2_MagIC_hysteresis/example_data/RMB_oxyhydroxides/measurements.txt


### Examine experiments

In [12]:
oxy_experiments = rmag.make_experiment_df(oxy_measurements)
oxy_experiments

,specimen,method_codes,experiment
0,ferroxyhyte_Princeton-1985-M1-1,LP-CW-SIRM:LP-MC,ferroxyhyte_Princeton-1985-M1-1-LP-CW-SIRM:LP-...
1,ferroxyhyte_Princeton-1985-M1-1,LP-CW-SIRM:LP-MW,ferroxyhyte_Princeton-1985-M1-1-LP-CW-SIRM:LP-...
2,ferroxyhyte_Princeton-1985-M1-1,LP-FC,ferroxyhyte_Princeton-1985-M1-1-LP-FC-DC-9652
3,ferroxyhyte_Princeton-1985-M1-1,LP-MST,ferroxyhyte_Princeton-1985-M1-1-LP-MST-DC-9652
4,ferroxyhyte_Princeton-1985-M1-1,LP-X:LP-X-T:LP-X-F,IRM-OldBlue-LP-X:LP-X-T:LP-X-F-9652
5,ferroxyhyte_Princeton-1985-M1-1,LP-ZFC,ferroxyhyte_Princeton-1985-M1-1-LP-ZFC-DC-9652
6,ferroxyhyte_Princeton-1985-M1-2,LP-X-T,IRM-KappaF-LP-X-T-3413
7,ferroxyhyte_Princeton-1985-M2-1,LP-CW-SIRM:LP-MC,ferroxyhyte_Princeton-1985-M2-1-LP-CW-SIRM:LP-...
8,ferroxyhyte_Princeton-1985-M2-1,LP-CW-SIRM:LP-MW,ferroxyhyte_Princeton-1985-M2-1-LP-CW-SIRM:LP-...
9,ferroxyhyte_Princeton-1985-M2-1,LP-FC,ferroxyhyte_Princeton-1985-M2-1-LP-FC-DC-9655


### Plot low-temperature data

In [23]:
specimen_name = 'goethite_Pfizer-YLO1888D-1'
fc_data, zfc_data, rtsirm_cool_data, rtsirm_warm_data = rmag.extract_mpms_data_dc(oxy_measurements, specimen_name)
rmag.plot_mpms_dc(fc_data, zfc_data, rtsirm_cool_data, rtsirm_warm_data, 
                  interactive=True, drop_first=True)

## Let's upload a contribution! 

1. Download this MagIC contribution which contains 


Log-in to MagIC/Earthref (https://www2.earthref.org/MagIC)
2. Go to upload tool (https://www2.earthref.org/MagIC/upload)
3. Download and examine contribution 

   <img src="images/download_contribution.png" width="500"/>
4. Go to Data model to learn table/column names (https://www2.earthref.org/MagIC/data-models/3.0)
5. Go back to main upload page and let's drag and drop the file to upload

   <img src="images/upload_data.png" width="500"/>
6. Upon uploading, we can examine the tables which are parsed and matched up with the table name and data model fields

   <img src="images/import_data.png" width="500"/>
7. Click the upload button in the lower right, which will bring you to an upload page where you can click through to upload data into your private workspace
   
   <img src="images/private_workspace.png" width="500"/>
8. Here we can add the DOI for the study which is `10.1029/93JB00024` (then click save), add the lab for the study `Paleomagnetic Laboratory (USGS Menlo Park, USA)`, and then click to validate the contribution
   
   <img src="images/manage_contribution.png" width="500"/>
9. Once the study is validated, a contribution can be published (let's not do that here as this study is already in MagIC) or for a private contribution a private link can be shared. Sharing such a link enables data to be made available for peer review. Note that the DOI for the contribution (Future Data DOI) is already available at this stage which enables the data to be properly referenced within a manuscript when it is submitted. 

   <img src="images/private_share.png" width="400"/>

### Download and unpack data directly from a private MagIC contribution

To bring in a data from a contribution that has been uploaded to MagIC, but that is not yet publically published, you need to both set the `magic_id` (```'20100'``` for the private contribution in this example) and provide the share key for your MagIC contribution which you can find by clicking on the "Share" button in the MagIC database. Here we will do that with data developed by participants in one of the groups from the IRM Summer School.

<figure>
  <img src="https://raw.githubusercontent.com/PmagPy/RockmagPy-notebooks/main/book/images/MagIC_private_contribution_1.png" alt="Description of the image" width="800">
</figure>

Copy the share key highlighted in grey in the following image to the `share_key` parameter in the cell below.

<figure>
  <img src="https://raw.githubusercontent.com/PmagPy/RockmagPy-notebooks/main/book/images/MagIC_private_contribution_2.png" alt="Description of the image" width="800">
</figure>

In [ ]:
# set the MagIC ID for the data set here
magic_id = ''

# provide the share key for the data set
share_key = ''

# set where you want the downloaded data to go
dir_path = 'example_data/my_data'

result, magic_file = ipmag.download_magic_from_id(magic_id, directory=dir_path, share_key=share_key)
ipmag.unpack_magic(magic_file, dir_path, print_progress=False)
my_contribution = cb.Contribution(dir_path)

In [31]:
# set the directory path (dir_path) to the data
dir_path = './example_data/SSRM'

# set the name of the MagIC file
ipmag.unpack_magic('example_MagIC_file.txt', 
                     dir_path = dir_path,
                     input_dir_path = dir_path,
                     print_progress=False)

SSRM_contribution = cb.Contribution(dir_path)
SSRM_measurements = SSRM_contribution.tables['measurements'].df

1  records written to file  /Users/penokean/0000_GitHub/2025_rockmagpy_workshop/2_MagIC_hysteresis/example_data/SSRM/locations.txt
7  records written to file  /Users/penokean/0000_GitHub/2025_rockmagpy_workshop/2_MagIC_hysteresis/example_data/SSRM/samples.txt
14  records written to file  /Users/penokean/0000_GitHub/2025_rockmagpy_workshop/2_MagIC_hysteresis/example_data/SSRM/specimens.txt
1  records written to file  /Users/penokean/0000_GitHub/2025_rockmagpy_workshop/2_MagIC_hysteresis/example_data/SSRM/sites.txt
3058  records written to file  /Users/penokean/0000_GitHub/2025_rockmagpy_workshop/2_MagIC_hysteresis/example_data/SSRM/measurements.txt


In [32]:
SSRM_experiments = rmag.make_experiment_df(SSRM_measurements)
SSRM_experiments

,specimen,method_codes,experiment
0,MA1-1b,LP-X:LP-X-F:LP-X-H,IRM-Magnon-LP-X:LP-X-F:LP-X-H-14466
1,MA1-2b,LP-X:LP-X-F:LP-X-H,IRM-Magnon-LP-X:LP-X-F:LP-X-H-14467
2,MA1-2x,LP-X-T,IRM-KappaF-LP-X-T-7724
3,MA1-3b,LP-X:LP-X-F:LP-X-H,IRM-Magnon-LP-X:LP-X-F:LP-X-H-14468
4,MA1-5-r gelcap,LP-BCR-BF,IRM-VSM3-LP-BCR-BF-239370
5,MA1-5-r gelcap,LP-CW-SIRM:LP-MC,MA1-5-r gelcap-LP-CW-SIRM:LP-MC-DC-14668
6,MA1-5-r gelcap,LP-CW-SIRM:LP-MW,MA1-5-r gelcap-LP-CW-SIRM:LP-MW-DC-14668
7,MA1-5-r gelcap,LP-FC,MA1-5-r gelcap-LP-FC-DC-14668
8,MA1-5-r gelcap,LP-HYS,IRM-VSM3-LP-HYS-239371
9,MA1-5-r gelcap,LP-ZFC,MA1-5-r gelcap-LP-ZFC-DC-14668


In [28]:
#specify the experiment name and specimen name
experiment_name = 'IRM-VSM3-LP-HYS-239371'
specimen_name = 'MA1-OX-r'

experiment_hyst = rmag.experiment_selection(SSRM_measurements,
                                            experiment_name)
experiment_results = rmag.process_hyst_loop(experiment_hyst['meas_field_dc'].values, 
                                            experiment_hyst['magn_mass'].values, specimen_name)

### Plot a thermomagnetic curve

In the cell below put in an experiment name associated with one of the LP-X-T experiments. 

In [34]:
experiment_name = 'IRM-KappaF-LP-X-T-7733'
selected_experiment = rmag.experiment_selection(SSRM_measurements,
                                                experiment_name)

rmag.plot_X_T(selected_experiment, 
              temp_unit='C', 
              remove_holder=True, 
              plot_derivative=True,
              plot_inverse=True,
              interactive=True,
              figsize=(8,3))

In [15]:
specimen_name = 'MA1-5-r gelcap'
fc_data, zfc_data, rtsirm_cool_data, rtsirm_warm_data = rmag.extract_mpms_data_dc(SSRM_measurements, specimen_name)
rmag.plot_mpms_dc(fc_data, zfc_data, rtsirm_cool_data, rtsirm_warm_data,
                  plot_derivative=True, interactive=True)

In [30]:
experiment_name = 'IRM-VSM4-LP-BCR-BF-239379'
selected_experiment = rmag.experiment_selection(SSRM_measurements,
                                                experiment_name)
processed_experiment, Bcr = rmag.backfield_data_processing(selected_experiment,
                                                           smooth_mode='spline',
                                                           smooth_frac=0.000005)
rmag.plot_backfield_data(processed_experiment, Bcr=Bcr, interactive=True)

In [20]:
selected_experiment

,citations,experiment,instrument_codes,meas_temp,measurement,method_codes,quality,sequence,specimen,standard,susc_chi_mass,timestamp
0,This study,IRM-KappaF-LP-X-T-7724,IRM-MPMS3,312.3,MA1-2x-IRM-KappaF-LP-X-T-7724-7724-89,LP-X-T,g,89.0,MA1-2x,u,-0.000006,2024-06-05T10:36:25.000-05:00
1,This study,IRM-KappaF-LP-X-T-7724,IRM-MPMS3,314.4,MA1-2x-IRM-KappaF-LP-X-T-7724-7724-89,LP-X-T,g,89.0,MA1-2x,u,-0.000006,2024-06-05T10:36:25.000-05:00
2,This study,IRM-KappaF-LP-X-T-7724,IRM-MPMS3,317.0,MA1-2x-IRM-KappaF-LP-X-T-7724-7724-89,LP-X-T,g,89.0,MA1-2x,u,-0.000006,2024-06-05T10:36:25.000-05:00
3,This study,IRM-KappaF-LP-X-T-7724,IRM-MPMS3,319.9,MA1-2x-IRM-KappaF-LP-X-T-7724-7724-89,LP-X-T,g,89.0,MA1-2x,u,-0.000006,2024-06-05T10:36:25.000-05:00
4,This study,IRM-KappaF-LP-X-T-7724,IRM-MPMS3,322.1,MA1-2x-IRM-KappaF-LP-X-T-7724-7724-89,LP-X-T,g,89.0,MA1-2x,u,-0.000006,2024-06-05T10:36:25.000-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...
447,This study,IRM-KappaF-LP-X-T-7724,IRM-MPMS3,330.0,MA1-2x-IRM-KappaF-LP-X-T-7724-7724-89,LP-X-T,g,89.0,MA1-2x,u,-0.000005,2024-06-05T10:36:25.000-05:00
448,This study,IRM-KappaF-LP-X-T-7724,IRM-MPMS3,327.7,MA1-2x-IRM-KappaF-LP-X-T-7724-7724-89,LP-X-T,g,89.0,MA1-2x,u,-0.000005,2024-06-05T10:36:25.000-05:00
449,This study,IRM-KappaF-LP-X-T-7724,IRM-MPMS3,325.7,MA1-2x-IRM-KappaF-LP-X-T-7724-7724-89,LP-X-T,g,89.0,MA1-2x,u,-0.000005,2024-06-05T10:36:25.000-05:00
450,This study,IRM-KappaF-LP-X-T-7724,IRM-MPMS3,323.5,MA1-2x-IRM-KappaF-LP-X-T-7724-7724-89,LP-X-T,g,89.0,MA1-2x,u,-0.000005,2024-06-05T10:36:25.000-05:00


In [7]:
selected_experiment

,citations,experiment,instrument_codes,meas_temp,measurement,method_codes,quality,sequence,specimen,standard,susc_chi_mass,timestamp
0,This study,IRM-KappaF-LP-X-T-7733,IRM-MPMS3,297.1,MA1-OX-r-IRM-KappaF-LP-X-T-7733-7733-89,LP-X-T,g,89.0,MA1-OX-r,u,-2.310000e-07,2024-06-12T08:56:32.000-05:00
1,This study,IRM-KappaF-LP-X-T-7733,IRM-MPMS3,300.3,MA1-OX-r-IRM-KappaF-LP-X-T-7733-7733-89,LP-X-T,g,89.0,MA1-OX-r,u,-2.840000e-07,2024-06-12T08:56:32.000-05:00
2,This study,IRM-KappaF-LP-X-T-7733,IRM-MPMS3,305.3,MA1-OX-r-IRM-KappaF-LP-X-T-7733-7733-89,LP-X-T,g,89.0,MA1-OX-r,u,-4.260000e-07,2024-06-12T08:56:32.000-05:00
3,This study,IRM-KappaF-LP-X-T-7733,IRM-MPMS3,310.9,MA1-OX-r-IRM-KappaF-LP-X-T-7733-7733-89,LP-X-T,g,89.0,MA1-OX-r,u,-4.970000e-07,2024-06-12T08:56:32.000-05:00
4,This study,IRM-KappaF-LP-X-T-7733,IRM-MPMS3,315.7,MA1-OX-r-IRM-KappaF-LP-X-T-7733-7733-89,LP-X-T,g,89.0,MA1-OX-r,u,-5.500000e-07,2024-06-12T08:56:32.000-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...
434,This study,IRM-KappaF-LP-X-T-7733,IRM-MPMS3,329.0,MA1-OX-r-IRM-KappaF-LP-X-T-7733-7733-89,LP-X-T,g,89.0,MA1-OX-r,u,-1.400000e-06,2024-06-12T08:56:32.000-05:00
435,This study,IRM-KappaF-LP-X-T-7733,IRM-MPMS3,327.2,MA1-OX-r-IRM-KappaF-LP-X-T-7733-7733-89,LP-X-T,g,89.0,MA1-OX-r,u,-1.390000e-06,2024-06-12T08:56:32.000-05:00
436,This study,IRM-KappaF-LP-X-T-7733,IRM-MPMS3,325.7,MA1-OX-r-IRM-KappaF-LP-X-T-7733-7733-89,LP-X-T,g,89.0,MA1-OX-r,u,-1.380000e-06,2024-06-12T08:56:32.000-05:00
437,This study,IRM-KappaF-LP-X-T-7733,IRM-MPMS3,324.1,MA1-OX-r-IRM-KappaF-LP-X-T-7733-7733-89,LP-X-T,g,89.0,MA1-OX-r,u,-1.380000e-06,2024-06-12T08:56:32.000-05:00


In [5]:
selected_experiment

,specimen,method_codes,experiment
0,MA1-OX-r,LP-X-T,IRM-KappaF-LP-X-T-7733


In [ ]:
experiment_name = 'IRM-VSM3-LP-HYS-228041'
DA4_r_hys = SSRM2022C_measurements[SSRM2022C_measurements['experiment'].str.contains(experiment_name, na=False)]
DA4_r_hys.dropna(axis=1, how='all', inplace=True)
DA4_r_hys.head()

### Isolate data from a single specimen

We can alternatively filter the measurements to get all the data for a specified specimen

In [ ]:
specimen_name = 'DX1-5r'
specimen_data = SSRM2022C_measurements[SSRM2022C_measurements["specimen"] == specimen_name]
specimen_data.dropna(axis=1, how='all', inplace=True)
specimen_data.head()

#### Filter specimen data on method code.

Filtering can also be done using the associated `method_code`.

In [ ]:
experiment_method_code = 'LP-FC'
fc_data = specimen_data[specimen_data['method_codes'].str.contains(experiment_method_code, na=False)]
fc_data.head()